In [34]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By 
from datetime import datetime
from PIL import Image
import pandas as pd
import pytesseract
import requests
import time
import glob
import re
import os

In [35]:
# Inicializar o WebDriver do Selenium (neste caso, usando o Chrome)

service = Service()

options = webdriver.ChromeOptions()

options.add_argument("--headless")

driver = webdriver.Chrome(service=service, options=options)

In [36]:
# Url do site
url = 'https://rpachallengeocr.azurewebsites.net/'

In [37]:
def fazer_download_links_pagina(idx):
    linhas_tabela = driver.find_elements(By.XPATH, '//tbody/tr')

    for linha in linhas_tabela:
        try:
            # Encontrar o link de download na linha atual
            link_download = linha.find_element(By.XPATH, './/td/a')

            # Encontrar a data na linha "Due Date"
            due_date = pd.to_datetime(linha.find_element(By.XPATH, './td[3]').text, dayfirst=True)

            # Verificar se o Invoice já venceu ou vence no dia de hoje
            if (due_date <= pd.to_datetime(datetime.today().date())):
                # Obter o URL do link de download
                url_download = link_download.get_attribute('href')

                # Fazer o download
                response = requests.get(url_download)

                # Verificar se a solicitação foi bem-sucedida
                if response.status_code == 200:
                    # Gerar um nome de arquivo exclusivo com um número sequencial
                    nome_arquivo = f"nome_do_arquivo_{idx}.jpg"
                    # Salvar o arquivo
                    with open(nome_arquivo, "wb") as f:
                        f.write(response.content)
                    print(f"Download do arquivo {nome_arquivo} concluído com sucesso!")
                    # Incrementar idx após o download de cada arquivo
                    idx += 1
                else:
                    print("Falha ao baixar o arquivo.")
            else:
                #Pular para a próxima linha da tabela
                idx += 1
                
        # Se ocorrer a exceção StaleElementReferenceException, continuamos para a próxima iteração
        except StaleElementReferenceException:
            continue

    # Retorna o próximo valor de idx
    return idx

# Inicialize o índice
idx = 1

In [38]:
# Extrai informaçoes da tabela e cria lista com elas
def extrai_dados_tabela(df_table):
    ids = []
    due_dates = []
    linhas_tabela = driver.find_elements(By.XPATH, '//tbody/tr')

    # Iterar sobre as linhas da tabela
    for linha in linhas_tabela:
        id = linha.find_element(By.XPATH, './td[2]').text
        due_date = linha.find_element(By.XPATH, './td[3]').text

        # Encontrar a data na linha "Due Date"
        due_date_num = pd.to_datetime(due_date, dayfirst=True)

        # Verificar se o Invoice já venceu ou vence no dia de hoje
        if due_date_num <= pd.to_datetime(datetime.today().date()):
            ids.append(id)
            due_dates.append(due_date)

    # Criar um DataFrame com os novos dados
    novos_dados = pd.DataFrame({'ID': ids, 'Due Date': due_dates})

    # Concatenar o novo DataFrame com o DataFrame existente
    df_table = pd.concat([df_table, novos_dados], ignore_index=True)

    return df_table

In [39]:
# Abrir a página inicial

driver.get(url)

In [40]:
# Define um DataFrame vazio
df_table = pd.DataFrame(columns=['ID', 'Due Date'])

# Loop para navegar por todas as páginas da tabela
while True:
    # Chamada da função fazer_download_links_pagina() com o índice atual
    idx = fazer_download_links_pagina(idx)
    # Chamada da função que extrai os dados da tebela do site
    df_table = extrai_dados_tabela(df_table)

    # verifica se o botão "Next" está desabilitado
    next_button = driver.find_element(By.ID, "tableSandbox_next")
    if "disabled" in next_button.get_attribute("class"):
        break

    next_button.click()

    time.sleep(0.1)
    
driver.quit()

Download do arquivo nome_do_arquivo_2.jpg concluído com sucesso!
Download do arquivo nome_do_arquivo_4.jpg concluído com sucesso!
Download do arquivo nome_do_arquivo_7.jpg concluído com sucesso!
Download do arquivo nome_do_arquivo_10.jpg concluído com sucesso!
Download do arquivo nome_do_arquivo_11.jpg concluído com sucesso!
Download do arquivo nome_do_arquivo_12.jpg concluído com sucesso!


In [41]:
# Lista para armazenar as informações extraídas de cada arquivo
informacoes_extraidas = []

# Função para extrair informações de um arquivo de texto
def extrair_informacoes_arquivo(texto_extraido):

    # Padrões de regex para extrair as informações desejadas
    padrao_invoice_number = r'#\s*(\d+)'
    padrao_invoice_date = r'(\w{3}\s+\d{1,2},\s+\d{4}|\d{4}-\d{2}-\d{2})'
    padrao_company_name = r'(Aenean\s*LLC|Sit\s*Amet\s*Corp)'
    padrao_total_due = r'(?:Total\s*:\s*\$?|Total\s*)([\d,.]+)'

    # Extrair as informações usando regex
    invoice_number_match = re.search(padrao_invoice_number, texto_extraido)
    invoice_date_match = re.search(padrao_invoice_date, texto_extraido)
    company_name_match = re.search(padrao_company_name, texto_extraido)
    total_due_match = re.search(padrao_total_due, texto_extraido)

    # Armazenar as informações extraídas
    informacoes = {}

    # Verificar se as informações foram encontradas e adicionar ao dicionário
    if invoice_number_match:
        informacoes['Invoice Number'] = invoice_number_match.group(1)
    if invoice_date_match:
        # Verificar o formato da data
        data = invoice_date_match.group(1)
        if re.match(r'\d{4}-\d{2}-\d{2}', data):
            # Se a data estiver no formato 'yyyy-mm-dd', converter para 'dd-mm-yyyy'
            data_formatada = datetime.strptime(data, '%Y-%m-%d').strftime('%d-%m-%Y')
            informacoes['Invoice Date'] = data_formatada
        elif re.match(r'\w{3}\s+\d{1,2},\s+\d{4}', data):
            # Se a data estiver no formato 'Jun 1, 2019', converter para 'dd-mm-yyyy'
            data_formatada = datetime.strptime(data, '%b %d, %Y').strftime('%d-%m-%Y')
            informacoes['Invoice Date'] = data_formatada
    if company_name_match:
        informacoes['Company Name'] = company_name_match.group(1)
    if total_due_match:
        informacoes['Total Due'] = total_due_match.group(1)
    # Verificar se o formato do Total Due é numérico com separador de milhares e decimais
    if total_due_match:
        total_due = total_due_match.group(1)
        if re.match(r'^\d{1,3}(,\d{3})*\.\d{2}$', total_due):
            # Se o formato for "21,442.80", converter para 4139.60
            total_due_formatado = '{:.2f}'.format(float(total_due.replace(',', '')))
            informacoes['Total Due'] = total_due_formatado
        else:
            informacoes['Total Due'] = total_due

    if informacoes:
        # Adicionar as informações extraídas à lista
        informacoes_extraidas.append(informacoes)
    else:
        print(f"Falha ao extrair informações do arquivo")

In [42]:
# Lista todos os arquivos .jpg no diretório atual e os ordena numericamente
arquivos_jpg = sorted(glob.glob('*.jpg'), key=lambda x: int(''.join(filter(str.isdigit, x))))

# Loop através de todos os arquivos .jpg encontrados
for arquivo in arquivos_jpg:
    # Abrir a imagem
    image = Image.open(arquivo)

    # Extrair texto da imagem usando o Tesseract OCR
    texto_extraido = pytesseract.image_to_string(image)

    extrair_informacoes_arquivo(texto_extraido)


In [43]:
# Criar um DataFrame com as informações extraídas
df_informacoes_extraidas = pd.DataFrame(informacoes_extraidas)

# Mesclar os DataFrames com base no índice
tabela_completa = pd.concat([df_informacoes_extraidas, df_table], axis=1)

# Salvar o DataFrame resultante em um novo arquivo CSV
tabela_completa.to_csv('tabela_completa.csv', index=False)

print("Informações extraídas:")
print(tabela_completa)


Informações extraídas:
  Invoice Number Invoice Date Company Name Total Due                      ID  \
0         284210   01-06-2019   Aenean LLC   3000.00  p7xb1lyp1gc1npaldfclot   
1         284213   03-06-2019   Aenean LLC   9778.40  sbo79920qjf2l0072993qx   
2         284221   20-06-2019   Aenean LLC   6300.00  eckfs24nvpkx62r593ek29   
3         284228   28-06-2019   Aenean LLC   1800.00  gtmtvj3iik4p1f4cipmdyq   
4         284232   15-06-2019   Aenean LLC   1009.80  z63ewe4x0hqt2qvxztcydd   
5         284212   02-06-2019   Aenean LLC   4139.60  hm9ffor26etjbzi3uss2jd   

     Due Date  
0  06-02-2024  
1  20-01-2024  
2  01-02-2024  
3  31-01-2024  
4  20-01-2024  
5  17-02-2024  


In [44]:
# Obtém o caminho do diretório atual do projeto
diretorio_projeto = os.getcwd()

# Padrão de busca para arquivos .jpg
padrao_arquivos_jpg = os.path.join(diretorio_projeto, '*.jpg')

# Lista todos os arquivos .jpg no diretório do projeto
arquivos_jpg = glob.glob(padrao_arquivos_jpg)

# Exclui cada arquivo .jpg encontrado
for arquivo in arquivos_jpg:
    os.remove(arquivo)

print("Todos os arquivos .jpg foram excluídos com sucesso.")

Todos os arquivos .jpg foram excluídos com sucesso.
